In [25]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import seaborn as sns

#Preprocessing
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
lmtsr = WordNetLemmatizer()
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
from wordcloud import WordCloud, STOPWORDS

#Sentiment Analysis
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax
from tqdm.notebook import tqdm
from textblob import TextBlob, Word, Blobber
from nltk.sentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

#T-test
from scipy.stats import ttest_ind
import scipy.stats as stats
import scipy.stats

from collections import defaultdict
import nltk
from nltk.corpus import stopwords

# Load the stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

#calender
import calendar
import statsmodels.api as sm
import statsmodels.formula.api as smf

#Modelling
import json
import glob
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
# spacy
import spacy
from nltk.corpus import stopwords
# vis
import pyLDAvis
import pyLDAvis.gensim

import ast
from sklearn.cluster import KMeans
import numpy as np

import plotly.express as px
import plotly.io as pio

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/matthewbaber/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [26]:
hf = pd.read_csv('/Users/matthewbaber/Library/Mobile Documents/com~apple~CloudDocs/1/Master/yr4/ARP/data/hanoi.csv')
dn = pd.read_csv('/Users/matthewbaber/Library/Mobile Documents/com~apple~CloudDocs/1/Master/yr4/ARP/data/da_nang.csv')
hcm = pd.read_csv('/Users/matthewbaber/Library/Mobile Documents/com~apple~CloudDocs/1/Master/yr4/ARP/data/hcm.csv')
ct = pd.read_csv('/Users/matthewbaber/Library/Mobile Documents/com~apple~CloudDocs/1/Master/yr4/ARP/data/cantho.csv')

#add column to each df to identify city
hf['city'] = 'Hanoi'
dn['city'] = 'Da Nang'
hcm['city'] = 'Ho Chi Minh City'
ct['city'] = 'Can Tho'

In [27]:
#concatenate all dataframes
df = pd.concat([ct, dn, hf, hcm])

def data_cleaning(df):

    #drop of nan that has a nan in links_n_review
    df = df.dropna(subset=['links_n_review', 'links_n_date'])

#split links_n_date into two columns on •
    df[['date', 'links']] = df['links_n_date'].str.split('•', expand=True)

    return df

def sorting_out_dates(df):

    #extract the year from the date column
    df['year'] = df['date'].str[-3:]

    def get_month_number(month):
        months = {
            'Jan': 1,
            'Feb': 2,
            'Mar': 3,
            'Apr': 4,
            'May': 5,
            'Jun': 6,
            'Jul': 7,
            'Aug': 8,
            'Sep': 9,
            'Oct': 10,
            'Nov': 11,
            'Dec': 12
        }
        return months[month]

    #use this to map the months in the date colum to a number
    df['month'] = df['date'].str[:3].apply(get_month_number)

    #combine the month and year columns
    df['DATE'] = df['month'].astype(str) +  ':' + df['year'].astype(str)

    #drop the redundant columns
    df = df.drop(columns=['year', 'month', 'date'])

    #remove '-' from the DATE column
    df['DATE'] = df['DATE'].str.replace('-', '')

    #add '20'after : in the DATE column
    df['DATE'] = df['DATE'].str.replace(':', '/20')

    #remove empty space in the DATE column
    df['DATE'] = df['DATE'].str.replace(' ', '')

    return df

def datetime(df):
    #add in datetime to the DATE column
    df['DATE'] = pd.to_datetime(df['DATE'], format='%m/%Y', errors='coerce')

    #reset index
    df = df.reset_index(drop=True)

    return df


df = data_cleaning(df)
df = sorting_out_dates(df)
df = datetime(df)

#delete duplicates
df = df.drop_duplicates(subset=['links_n_review_title_url'], keep='first')

timeline = df['DATE'].unique()

pivot = df.copy()

def pivot_table(pivot):
    #extract the first 4 numbers of the DATE column
    pivot['year'] = pivot['DATE'].astype(str).str[:4]

    #check in the city column if the city is Hanoi, then create a new column with 1 or 0
    pivot['Hanoi'] = np.where(pivot['city'] == 'Hanoi', 1, 0)
    pivot['Can Tho'] = np.where(pivot['city'] == 'Can Tho', 1, 0)
    pivot['Da Nang'] = np.where(pivot['city'] == 'Da Nang', 1, 0)
    pivot['Ho Chi Minh City'] = np.where(pivot['city'] == 'Ho Chi Minh City', 1, 0)

    pivot = pivot.pivot_table(index=['year'], values=['Hanoi', 'Can Tho', 'Da Nang', 'Ho Chi Minh City'], aggfunc='sum')

    return pivot

pivottable = pivot_table(pivot)
pivottable

/var/folders/rh/dpkch0xj1dx4bbcqs_mnb3qr0000gn/T/ipykernel_41274/1508459348.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['date', 'links']] = df['links_n_date'].str.split('•', expand=True)
/var/folders/rh/dpkch0xj1dx4bbcqs_mnb3qr0000gn/T/ipykernel_41274/1508459348.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['date', 'links']] = df['links_n_date'].str.split('•', expand=True)


,Can Tho,Da Nang,Hanoi,Ho Chi Minh City
year,,,,
2011,0,0,0,1
2012,0,3,0,3
2013,1,8,6,15
2014,20,24,69,82
2015,60,174,160,332
2016,64,431,415,1047
2017,82,683,747,1207
2018,70,952,1267,1948
2019,215,2043,2737,2709


In [28]:
stop_words = set(stopwords.words('english'))

def data_cleaning(df):

#clean the text in review column by removing any [~.,%/:;?_&+*=!-]
    def preprocess_text(text):
        cleaned = re.sub(r'[~.,%/:;?_&+*=!-]', '', text)
        return cleaned

    df['links_n_review'] = df['links_n_review'].apply(preprocess_text)

    #tokenization
    from nltk import word_tokenize,sent_tokenize 
    df['text_t'] = df['links_n_review'].str.lower().apply(word_tokenize)

    #stopwords removal
    df['text_t'] = df['text_t'].apply(lambda x: [word for word in x if word.isalpha() and word not in stop_words])

    #lemmatization
    df["text_lmt"] = df["text_t"].apply(lambda x: [lmtsr.lemmatize(word) for word in x])

    #stemming
    ps = PorterStemmer()
    df["text_stm"] = df["text_t"].apply(lambda x: [ps.stem(word) for word in x])

    df['text_stm'] = df['text_stm'].astype(str)
        
    df[".len_text"] = df["text_lmt"].str.len()
    return df

df.reset_index(drop=True, inplace=True)

df = data_cleaning(df)
df

,links_destination,links_page,links_n_contributions,links_n_review_title,links_n_review_title_url,links_n_review,links_n_date_written,links_n_date,city,links,DATE,text_t,text_lmt,text_stm,.len_text
0,Cai Rang Floating Market,"Showing results 201-210 of 1,186",5 contributions,Best trip ever,https://www.tripadvisor.co.uk/ShowUserReviews-...,Our family booked the trip to market via Then ...,Written 18 July 2023,Jun 2023 • Family,Can Tho,Family,2023-06-01,"[family, booked, trip, market, via, organised,...","[family, booked, trip, market, via, organised,...","['famili', 'book', 'trip', 'market', 'via', 'o...",97
1,Cai Rang Floating Market,"Showing results 201-210 of 1,186",9 contributions,Cai Rang floating market,https://www.tripadvisor.co.uk/ShowUserReviews-...,We visited Cai Rang floating market when join ...,Written 3 June 2023,May 2023 • Couples,Can Tho,Couples,2023-05-01,"[visited, cai, rang, floating, market, join, c...","[visited, cai, rang, floating, market, join, c...","['visit', 'cai', 'rang', 'float', 'market', 'j...",17
2,Cai Rang Floating Market,"Showing results 201-210 of 1,186",1 contribution,Wonderful Experience,https://www.tripadvisor.co.uk/ShowUserReviews-...,The market is very interesting with many fruit...,Written 8 May 2023,Apr 2023 • Couples,Can Tho,Couples,2023-04-01,"[market, interesting, many, fruit, boats, floa...","[market, interesting, many, fruit, boat, float...","['market', 'interest', 'mani', 'fruit', 'boat'...",31
3,Cai Rang Floating Market,"Showing results 201-210 of 1,186",1 contribution,Just amazing!!!,https://www.tripadvisor.co.uk/ShowUserReviews-...,Our guide picked up from our hotel and we got ...,Written 8 May 2023,May 2023 • Friends,Can Tho,Friends,2023-05-01,"[guide, picked, hotel, got, floating, market, ...","[guide, picked, hotel, got, floating, market, ...","['guid', 'pick', 'hotel', 'got', 'float', 'mar...",77
4,Cai Rang Floating Market,"Showing results 201-210 of 1,186",135 contributions,Good market experience,https://www.tripadvisor.co.uk/ShowUserReviews-...,Mr Toan from Boutique Homestay took us in his ...,Written 28 April 2023,Mar 2023 • Couples,Can Tho,Couples,2023-03-01,"[mr, toan, boutique, homestay, took, us, boat,...","[mr, toan, boutique, homestay, took, u, boat, ...","['mr', 'toan', 'boutiqu', 'homestay', 'took', ...",66
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24965,Museum of Vietnamese History,Showing results 421-430 of 582,603 contributions,Nice museum that gives a flavour of the city,https://www.tripadvisor.co.uk/ShowUserReviews-...,Museum is set in some nice grounds and gives a...,Written 15 June 2014,Jun 2014 • Couples,Ho Chi Minh City,Couples,2014-06-01,"[museum, set, nice, grounds, gives, potted, hi...","[museum, set, nice, ground, give, potted, hist...","['museum', 'set', 'nice', 'ground', 'give', 'p...",21
24966,Museum of Vietnamese History,Showing results 421-430 of 582,111 contributions,Overview of entire known history,https://www.tripadvisor.co.uk/ShowUserReviews-...,It's a little threadbare but the collection ra...,Written 12 June 2014,May 2014 • Couples,Ho Chi Minh City,Couples,2014-05-01,"[little, threadbare, collection, ranges, diora...","[little, threadbare, collection, range, dioram...","['littl', 'threadbar', 'collect', 'rang', 'dio...",44
24967,Museum of Vietnamese History,Showing results 421-430 of 582,52 contributions,Museum,https://www.tripadvisor.co.uk/ShowUserReviews-...,This was a great place lots of historical arti...,Written 8 June 2014,May 2014 • Couples,Ho Chi Minh City,Couples,2014-05-01,"[great, place, lots, historical, artifacts, mu...","[great, place, lot, historical, artifact, muse...","['great', 'place', 'lot', 'histor', 'artifact'...",14
24968,Museum of Vietnamese History,Showing results 421-430 of 582,3 contributions,cool place,https://www.tripadvisor.co.uk/ShowUserReviews-...,Loved this placeits inexpensive and staff seem...,Written 4 June 2014,May 2014 • Friends,Ho Chi Minh City,Friends,2014-05-01,"[loved, placeit

In [29]:
df = df[df['DATE'] >= '2023-01-01']

In [30]:
#bag of words and TF-IDF
bag_of_words = vectorizer.fit_transform(df['text_stm'])

feature_names = vectorizer.get_feature_names_out()

# create a new dataframe with the bag of words representation
tf = pd.DataFrame(bag_of_words.toarray(), columns = vectorizer.get_feature_names_out())
tfidf_sum = tf.T.sum(axis=1).sort_values(ascending = False).reset_index().rename(columns={"index": "word", 0: "tfidf_sum"})
tfidf_transposed = tf.T

tfidf_sum = tf.T.sum(axis=1).sort_values(ascending = False).reset_index().rename(columns={"index": "word", 0: "tfidf_sum"})

#find the 2000 most infrequent words 
infreq_sum = pd.DataFrame(tfidf_sum[-2500:])
infrequent_words = infreq_sum['word'].tolist()
infrequent_words

['field',
 'intro',
 'expand',
 'distinct',
 'adjac',
 'đức',
 'templew',
 'musuem',
 'importantli',
 'emperor',
 'punctual',
 'passerbi',
 'refil',
 'spark',
 'compliment',
 'lasiesta',
 'nghe',
 'prep',
 'souvernior',
 'psd',
 'noteworthi',
 'ukrain',
 'thien',
 'transpar',
 'alida',
 'minim',
 'upwel',
 'effortloc',
 'nva',
 'workout',
 'jw',
 'beforeaft',
 'okey',
 'snail',
 'apricot',
 'desecr',
 'guang',
 'purs',
 'previou',
 'herbarium',
 'ving',
 'hardwork',
 'maddi',
 'bodywork',
 'rolex',
 'hanoii',
 'procedur',
 'depart',
 'freaki',
 'oldschool',
 'dam',
 'alo',
 'summari',
 'whatev',
 'abund',
 'disquiet',
 'lossen',
 'arapang',
 'parallel',
 'pillar',
 'hype',
 'dug',
 'goid',
 'everysingledollar',
 'hole',
 'hint',
 'proffession',
 'helipad',
 'magpi',
 'devote',
 'remant',
 'defint',
 'woth',
 'unseen',
 'terrain',
 'ohh',
 'amazingw',
 'penisulaand',
 'marketssight',
 'brave',
 'colorfulhappi',
 'task',
 'assign',
 'stiff',
 'cam',
 'spotlight',
 'cheesi',
 'fortitud',


In [31]:
more_stop = ['n', 'de', 'u', 'le', 'ly', 'lo', 'ta', 'go', 'get' 'also', 'lot', 'great',
 'see',
 'good',
 'nice',
 'go',
 'worth',
 'lot',
 'well',
 'realli',
 'one',
 'get',
 'around',
 'vietnam',
 'mani',
 'enjoy',
 'hanoi',
 'experi',
 'love',
 'take',
 'would',
 'also',
 'day',
 'look',
 'amaz',
 'like',
 'vietnames',
 'much',
 'area',
 'even',
 'us',
 'come',
 'must',
 'hour',
 'way',
 'went',
 'best',
 'back',
 'definit',
 'make',
 'want',
 'thing',
 'close',
 'saigon',
 'chi',
 'locat',
 'littl',
 'could',
 'outsid',
 'small',
 'ho',
 'find',
 'need',
 'fun',
 'still',
 'bit',
 'excel',
 'da',
 'part',
 'tri',
 'nang',
 'feel',
 'minh',
 'long',
 'use',
 'better',
 'top',
 'sure',
 'minut',
 'first',
 'open',
 'got',
 'found',
 'end',
 'help',
 'floor',
 'everyth',
 'big',
 'spend',
 'stop',
 'took',
 'made',
 'along',
 'though',
 'side',
 'full',
 'everi',
 'especi',
 'give',
 'pretti',
 'easi',
 'know',
 'right',
 'noth',
 'howev',
 'fantast',
 'expect',
 'think',
 'cool',
 'whole',
 'next',
 'danang']

In [37]:
preprocessed_texts = df['text_stm']

preprocessed_texts = preprocessed_texts[~preprocessed_texts.isna()]

# Convert string representation of list to actual list of tokens
data_words = [ast.literal_eval(text) for text in preprocessed_texts]

# Step 2: Create a dictionary
id2word = corpora.Dictionary(data_words)

stop_words = ['ho', 'chi', 'minh', 'saigon', 'hanoi', 'hcmc', 'nguyen', 'vietnam', 'hcm', 'vietnames', 'da', 'nang', 'also','go', 'lot', 'ok'] + more_stop + infrequent_words

unwanted_word_ids = [id2word.token2id[word] for word in stop_words if word in id2word.token2id]
id2word.filter_tokens(bad_ids=unwanted_word_ids)

# Step 3: Convert text data to bag-of-words format
corpus = [id2word.doc2bow(text) for text in data_words]

# Step 4: Save corpus and dictionary to disk
corpora.MmCorpus.serialize('corpus.mm', corpus)
id2word.save('dictionary.gensim')

lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=id2word,
                                            num_topics=20,
                                            random_state=100,
                                            update_every=1,
                                            chunksize=100,
                                            passes=10,
                                            alpha='auto')

In [38]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds='mmds', R=15)
vis

/Users/matthewbaber/Library/Python/3.9/lib/python/site-packages/pyLDAvis/_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/Users/matthewbaber/Library/Python/3.9/lib/python/site-packages/sklearn/manifold/_mds.py:299: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
10    -0.449541 -0.222295       1        1  15.842406
18    -0.501641  0.061111       2        1  12.487465
4     -0.037052 -0.477474       3        1  10.044129
2     -0.234505 -0.377671       4        1   8.272951
9      0.057841  0.483833       5        1   7.298487
11    -0.249630  0.411447       6        1   6.945400
13    -0.326936  0.215886       7        1   4.218563
14    -0.211354 -0.191781       8        1   4.134107
1      0.172823 -0.402600       9        1   3.776352
12     0.391616 -0.227388      10        1   3.593433
7      0.170251 -0.256286      11        1   3.428225
19     0.318660  0.318366      12        1   3.201077
15    -0.209751  0.039655      13        1   3.023105
16    -0.031575 -0.055640      14        1   2.585561
8     -0.070249  0.257615      15        1   2.242448
3      0.110843  0.271464      16        1   2.095042
0      0.215174 -0.068850      17        1   2.048216
5      0.436158 -0.002284      18        1   2.045121
6      0.316058  0.127374      19        1   1.456900
17     0.132808  0.095518      20        1   1.261011, topic_info=           Term         Freq        Total Category  logprob  loglift
283       visit  1490.000000  1490.000000  Default  15.0000  15.0000
191       place  1017.000000  1017.000000  Default  14.0000  14.0000
10       massag  1209.000000  1209.000000  Default  13.0000  13.0000
7     recommend  1037.000000  1037.000000  Default  12.0000  12.0000
30         time   758.000000   758.000000  Default  11.0000  11.0000
...         ...          ...          ...      ...      ...      ...
1635        guy     0.070143     1.527913  Topic20  -9.1089   1.2921
1039      giang     0.070128     1.527718  Topic20  -9.1091   1.2920
1014       rang     0.070133     1.527873  Topic20  -9.1090   1.2920
2749      shirt     0.070135     1.527920  Topic20  -9.1090   1.2920
843       world     0.070140     1.528158  Topic20  -9.1090   1.2919

[333 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
19        2  0.985582   absolut
0         4  0.977323     activ
386       9  0.991614    actual
604      13  0.974713    almost
757      15  0.951729  although
...     ...       ...       ...
136       2  0.993277    welcom
310      18  0.886408    whilst
472       7  0.989009   without
60       14  0.992751    wonder
53       15  0.988171      work

[306 rows x 3 columns], R=15, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[11, 19, 5, 3, 10, 12, 14, 15, 2, 13, 8, 20, 16, 17, 9, 4, 1, 6, 7, 18])

# gensim

In [34]:
df = df.sort_values(by=['DATE'], ascending=False)

In [35]:
from gensim.corpora import Dictionary
from gensim.models import LdaModel, LdaSeqModel
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
from tqdm import tqdm

preprocessed_texts = df['text_stm']

preprocessed_texts = preprocessed_texts[~preprocessed_texts.isna()]

# Convert string representation of list to actual list of tokens
data_words = [ast.literal_eval(text) for text in preprocessed_texts]

# Step 2: Create a dictionary
id2word = corpora.Dictionary(data_words)

stop_words = ['ho', 'chi', 'minh', 'saigon', 'hanoi', 'hcmc', 'nguyen', 'vietnam', 'hcm', 'vietnames', 'da', 'nang'] + more_stop + infrequent_words

unwanted_word_ids = [id2word.token2id[word] for word in stop_words if word in id2word.token2id]
id2word.filter_tokens(bad_ids=unwanted_word_ids)

# Step 3: Convert text data to bag-of-words format
corpus = [id2word.doc2bow(text) for text in data_words]

# Step 4: Save corpus and dictionary to disk
corpora.MmCorpus.serialize('corpus.mm', corpus)
id2word.save('dictionary.gensim')

# Train an LDA model
lda_results = LdaModel(corpus=corpus, id2word=id2word, num_topics=2)


In [36]:
# Define time slices
time_slice = [202, 202, 202, 205]

# Initialize the LdaSeqModel with the pre-trained LDA model
ldaseq = LdaSeqModel(corpus=tqdm(corpus), time_slice=time_slice, num_topics=2, chunksize=1, lda_model=lda)

  0%|          | 0/2705 [00:00<?, ?it/s]


NameError: name 'lda' is not defined

In [ ]:
from gensim.test.utils import common_corpus, common_dictionary
from gensim.models import LdaSeqModel

# Define time slices
time_slice = [202, 202,]

# Initialize the model
ldaseq = LdaSeqModel(corpus=common_corpus, time_slice=time_slice, num_topics=2, chunksize=1)

# Print topics for each time period
for t in range(len(time_slice)):
    print(f"Time Period {t+1}:")
    topics = ldaseq.print_topics(time=t)
    for topic in topics:
        print([(common_dictionary[int(word_id)], weight) for word_id, weight in topic])


/Users/matthewbaber/Library/Python/3.9/lib/python/site-packages/gensim/models/ldaseqmodel.py:297: RuntimeWarning:

divide by zero encountered in scalar divide



Time Period 1:
[('user', 0.19909413177355004), ('response', 0.1406864104953201), ('survey', 0.1406864104953201), ('time', 0.1406864104953201), ('system', 0.1406864086643985), ('interface', 0.03963459288416942), ('eps', 0.03963459288416942), ('graph', 0.03963459288416942), ('minors', 0.03963459288416942), ('computer', 0.03377953134368177), ('human', 0.022921162597865835), ('trees', 0.022921162597865835)]
[('trees', 0.2412840094475041), ('system', 0.1695705721124763), ('human', 0.16957056076383753), ('graph', 0.16957056076383753), ('interface', 0.041107925485842384), ('eps', 0.041107925485842384), ('minors', 0.041107925485842384), ('computer', 0.03859062425800689), ('response', 0.022028867199674618), ('survey', 0.022028867199674618), ('time', 0.022028867199674618), ('user', 0.022003294597786733)]
Time Period 2:
[('user', 0.19918421423031168), ('system', 0.14073948538151645), ('response', 0.14073948430421418), ('survey', 0.14073948430421418), ('time', 0.14073948430421418), ('interface', 0

In [ ]:
data = df['links_n_review'].tolist()
data_ready = df['text_stm'].tolist()

In [ ]:
df.reset_index(drop=True, inplace=True)

In [ ]:
from collections import Counter

In [ ]:
dtngp = dtn.groupby(['Dominant_Topic', "Keywords", 'DATE', 'new_word']).size().reset_index(name='counts')
dtngp

,Dominant_Topic,Keywords,DATE,new_word,counts
0,0,"room, understand, anyth, photograph, huge, imp...",2023-04-01,"symbol, monasteri, sit, main, construct",1
1,2,"tour, guid, inform, book, souvenir, sell, pay,...",2020-02-01,"card, help, anoth, stop, say",1
2,2,"tour, guid, inform, book, souvenir, sell, pay,...",2023-03-01,"ticket, buy, elev, follow, take",1
3,5,"visit, market, time, interest, build, food, st...",2020-01-01,"visit, place, see, great, walk",1021
4,5,"visit, market, time, interest, build, food, st...",2020-02-01,"visit, place, see, worth, time",1081
5,5,"visit, market, time, interest, build, food, st...",2020-03-01,"visit, see, place, interest, war",571
6,5,"visit, market, time, interest, build, food, st...",2020-04-01,"walk, architectur, visit, love, stop",6
7,5,"visit, market, time, interest, build, food, st...",2020-05-01,"visit, vietnam, war, great, one",33
8,5,"visit, market, time, interest, build, food, st...",2020-06-01,"massag, beauti, easi, great, hot",16
9,5,"visit, market, time, interest, build, food, st...",2020-07-01,"time, good, visit, place, great",22


break

# ignore

In [ ]:
df_dominant_topic2 = df_dominant_topic

In [ ]:
#merge df_dominant_topic2 with lda2
lda2.reset_index(drop=True, inplace=True)  
df_dominant_topic2.reset_index(drop=True, inplace=True)
df_topic2 = pd.concat([lda2, df_dominant_topic2], axis=1)

In [ ]:
df_dominant_topic1 = df_dominant_topic

In [ ]:
#merge df_dominant_topic2 with lda2
lda1.reset_index(drop=True, inplace=True)  
df_dominant_topic1.reset_index(drop=True, inplace=True)
df_topic1 = pd.concat([lda1, df_dominant_topic1], axis=1)

In [ ]:
#concate df_topic1 and df_topic2
df_topic = pd.concat([df_topic1, df_topic2], axis=0)

Break

In [ ]:
dtn = pd.DataFrame()

#defining months and dates from date time to integer for the dataframe titles
months = ['january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december']

for year in range(2018, 2024):
    # Generate dates for the year
    dates = [f'{year}-01-01', f'{year}-02-01', f'{year}-03-01', f'{year}-04-01', f'{year}-05-01', f'{year}-06-01', f'{year}-07-01', f'{year}-08-01', f'{year}-09-01', f'{year}-10-01', f'{year}-11-01', f'{year}-12-01']
    
    #Separating the dataframe by month and then topic
    by_months = {}

    # Filter the data for each month and store it in the dictionary
    for month, date in zip(months, dates):
        by_months[month] = res[res['DATE'] == date]
    
        by_topics = {}

    # Iterate through the by_months dictionary
    for month, df in by_months.items():

        # Group the data by the dominant topic
        grouped = df.groupby('Dominant_Topic')
        
        # Iterate through the grouped data and store it in the by_topics dictionary
        for topic, group in grouped:
            by_topics[f'{month}{topic}'] = group

    # Loop through each topic in each month
    for key, df in by_topics.items():
        # Create the bag of words representation
        bag_of_words = vectorizer.fit_transform(df['text_stm'])
        feature_names = vectorizer.get_feature_names_out()
        # Create a new dataframe with the bag of words representation
        tf = pd.DataFrame(bag_of_words.toarray(), columns=feature_names)
        tfidf_sum = tf.T.sum(axis=1).sort_values(ascending=False).reset_index().rename(columns={"index": "word", 0: "tfidf_sum"})
        
        # Take the first 5 words of tfidf_sum and put them in a list called list_values
        list_values = tfidf_sum['word'][:5]
        comma_separated_string = ', '.join(list_values)
        
        # Update the new_word column in the by_topics dictionary
        by_topics[key]['new_word'] = [comma_separated_string] * len(df)

        dtn = pd.concat([dtn, by_topics[key]], ignore_index=True)

In [ ]:
dtngrouped = dtn.groupby(['DATE', 'Keywords', 'Dominant_Topic', 'new_word']).count().reset_index()

In [ ]:
vis = df_topic.groupby(['Keywords', 'DATE']).count().reset_index()

In [ ]:
dtn = pd.DataFrame()

for year in range(2023, 2024):
    # Generate dates for the year

    dates = [f'{year}-01-01', f'{year}-02-01', f'{year}-03-01', f'{year}-04-01', f'{year}-05-01', f'{year}-06-01', f'{year}-07-01', f'{year}-08-01', f'{year}-09-01', f'{year}-10-01', f'{year}-11-01', f'{year}-12-01']

    #defining months and dates from date time to integer for the dataframe titles
    months = ['january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december']

    #Separating the dataframe by month and then topic
    by_months = {}

    # Filter the data for each month and store it in the dictionary
    for month, date in zip(months, dates):
        by_months[month] = res[res['DATE'] == date]

        # Loop through each month and apply the TF-IDF function
        for month in months:
            # Create the bag of words representation
            bag_of_words = vectorizer.fit_transform(by_months[month]['Keywords'])
            feature_names = vectorizer.get_feature_names_out()

            # Create a new dataframe with the bag of words representation
            tf = pd.DataFrame(bag_of_words.toarray(), columns=feature_names)
            tfidf_sum = tf.T.sum(axis=1).sort_values(ascending=False).reset_index().rename(columns={"index": "word", 0: "tfidf_sum"})

            # Take the first 5 words of tfidf_sum and put them in a list called list_values
            list_values = tfidf_sum['word'][:5]
            comma_separated_string = ', '.join(list_values)

            # Update the new_word column in the by_months dictionary
            by_months[month]['new_word'] = [comma_separated_string] * len(by_months[month])

            # Concatenate the dataframes
            dtn = pd.concat([dtn, by_months[month]], ignore_index=True)

/var/folders/rh/dpkch0xj1dx4bbcqs_mnb3qr0000gn/T/ipykernel_20325/3311808641.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  by_months[month]['new_word'] = [comma_separated_string] * len(by_months[month])


KeyError: 'february'

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the TF-IDF vectorizer
vectorizer = TfidfVectorizer()

# Create an empty dataframe
dtn = pd.DataFrame()

# Define the months and dates
months = ['january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december']

# Create a dictionary to store the dataframes for each month
by_months = {}

# Filter the data for each month and store it in the dictionary
for year in range(2018, 2024):
    dates = [f'{year}-01-01', f'{year}-02-01', f'{year}-03-01', f'{year}-04-01', f'{year}-05-01', f'{year}-06-01', f'{year}-07-01', f'{year}-08-01', f'{year}-09-01', f'{year}-10-01', f'{year}-11-01', f'{year}-12-01']
    for month, date in zip(months, dates):
        by_months[month] = res[res['DATE'] == date]

    # Loop through each month and apply the TF-IDF function
    for month in months:
          if by_months[month].empty:
            print(f"No data found for {month}. Skipping.")
            continue
          # Create the bag of words representation
          bag_of_words = vectorizer.fit_transform(by_months[month]['Keywords'])
          feature_names = vectorizer.get_feature_names_out()

          # Create a new dataframe with the bag of words representation
          tf = pd.DataFrame(bag_of_words.toarray(), columns=feature_names)
          tfidf_sum = tf.T.sum(axis=1).sort_values(ascending=False).reset_index().rename(columns={"index": "word", 0: "tfidf_sum"})

          # Take the first 5 words of tfidf_sum and put them in a list called list_values
          list_values = tfidf_sum['word'][:5]
          comma_separated_string = ', '.join(list_values)

          # Update the new_word column in the by_months dictionary
          by_months[month]['new_word'] = [comma_separated_string] * len(by_months[month])

          # Concatenate the dataframes
          dtn = pd.concat([dtn, by_months[month]], ignore_index=True)

# good

In [39]:
data = df['links_n_review'].tolist()
data_ready = data_words

In [40]:
def format_topics_sentences(ldamodel=None, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data_ready)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
df_dominant_topic#.groupby('Keywords').sum().sort_values(by='Topic_Perc_Contrib', ascending=False)

/var/folders/rh/dpkch0xj1dx4bbcqs_mnb3qr0000gn/T/ipykernel_41274/745721134.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
/var/folders/rh/dpkch0xj1dx4bbcqs_mnb3qr0000gn/T/ipykernel_41274/745721134.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
/var/folders/rh/dpkch0xj1dx4bbcqs_mnb3qr0000gn/T/ipykernel_41274/745721134.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,2,0.2250,"food, market, understand, knowledg, learnt, en...","[great, activ, siri, deserv, bonu, would, reco..."
1,1,18,0.2906,"massag, recommend, highli, servic, staff, spa,...","[servic, amaz, massag, amaz, beyond, expect, r..."
2,2,18,0.2576,"massag, recommend, highli, servic, staff, spa,...","[got, perfect, servic, massag, hereand, peopl,..."
3,3,1,0.2297,"street, old, shop, actual, quarter, stay, impr...","[spa, ho, chi, minh, da, nang, one, best, one,..."
4,4,18,0.2202,"massag, recommend, highli, servic, staff, spa,...","[beauti, venu, love, massag, felt, relax, fini..."
...,...,...,...,...,...
2700,2700,4,0.1691,"guid, spent, differ, build, coupl, buy, travel...","[senior, coupl, late, feel, worth, go, tunnel,..."
2701,2701,10,0.3166,"visit, histori, learn, interest, beauti, citi,...","[visit, tour, group, reason, knowledg, tunnel,..."
2702,2702,10,0.2942,"visit, histori, learn, interest, beauti, citi,...","[nice, place, visit, half, day, ho, chi, minh,..."
2703,2703,18,0.2266,"massag, recommend, highli, servic, staff, spa,...","[eyeopen, peopl, live, fought, tunnel, highli,..."


In [41]:
# Merge on the index
new = pd.merge(df_dominant_topic, df, left_index=True, right_index=True)
new

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text,links_destination,links_page,links_n_contributions,links_n_review_title,links_n_review_title_url,links_n_review,links_n_date_written,links_n_date,city,links,DATE,text_t,text_lmt,text_stm,.len_text
0,0,2,0.2250,"food, market, understand, knowledg, learnt, en...","[great, activ, siri, deserv, bonu, would, reco...",Cai Rang Floating Market,"Showing results 201-210 of 1,186",5 contributions,Best trip ever,https://www.tripadvisor.co.uk/ShowUserReviews-...,Our family booked the trip to market via Then ...,Written 18 July 2023,Jun 2023 • Family,Can Tho,Family,2023-06-01,"[family, booked, trip, market, via, organised,...","[family, booked, trip, market, via, organised,...","['famili', 'book', 'trip', 'market', 'via', 'o...",97
1,1,18,0.2906,"massag, recommend, highli, servic, staff, spa,...","[servic, amaz, massag, amaz, beyond, expect, r...",Cai Rang Floating Market,"Showing results 201-210 of 1,186",9 contributions,Cai Rang floating market,https://www.tripadvisor.co.uk/ShowUserReviews-...,We visited Cai Rang floating market when join ...,Written 3 June 2023,May 2023 • Couples,Can Tho,Couples,2023-05-01,"[visited, cai, rang, floating, market, join, c...","[visited, cai, rang, floating, market, join, c...","['visit', 'cai', 'rang', 'float', 'market', 'j...",17
2,2,18,0.2576,"massag, recommend, highli, servic, staff, spa,...","[got, perfect, servic, massag, hereand, peopl,...",Cai Rang Floating Market,"Showing results 201-210 of 1,186",1 contribution,Wonderful Experience,https://www.tripadvisor.co.uk/ShowUserReviews-...,The market is very interesting with many fruit...,Written 8 May 2023,Apr 2023 • Couples,Can Tho,Couples,2023-04-01,"[market, interesting, many, fruit, boats, floa...","[market, interesting, many, fruit, boat, float...","['market', 'interest', 'mani', 'fruit', 'boat'...",31
3,3,1,0.2297,"street, old, shop, actual, quarter, stay, impr...","[spa, ho, chi, minh, da, nang, one, best, one,...",Cai Rang Floating Market,"Showing results 201-210 of 1,186",1 contribution,Just amazing!!!,https://www.tripadvisor.co.uk/ShowUserReviews-...,Our guide picked up from our hotel and we got ...,Written 8 May 2023,May 2023 • Friends,Can Tho,Friends,2023-05-01,"[guide, picked, hotel, got, floating, market, ...","[guide, picked, hotel, got, floating, market, ...","['guid', 'pick', 'hotel', 'got', 'float', 'mar...",77
4,4,18,0.2202,"massag, recommend, highli, servic, staff, spa,...","[beauti, venu, love, massag, felt, relax, fini...",Cai Rang Floating Market,"Showing results 201-210 of 1,186",135 contributions,Good market experience,https://www.tripadvisor.co.uk/ShowUserReviews-...,Mr Toan from Boutique Homestay took us in his ...,Written 28 April 2023,Mar 2023 • Couples,Can Tho,Couples,2023-03-01,"[mr, toan, boutique, homestay, took, us, boat,...","[mr, toan, boutique, homestay, took, u, boat, ...","['mr', 'toan', 'boutiqu', 'homestay', 'took', ...",66
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2653,2653,9,0.1394,"war, museum, french, quit, ride, prison, step,...","[advertis, super, cheap, given, valu, full, da...",3D Museum Art in Paradise Danang,Showing results 201-210 of 211,104 contributions,Good place for instagrammers and small kids,https://www.tripadvisor.co.uk/ShowUserReviews-...,We went there is match 2023 It was almost empt...,Written 30 March 2023,Mar 2023 • Family,Da Nang,Family,2023-03-01,"[went, match, almost, empty, get, enough, spac...","[went, match, almost, empty, get, enough, spac...","['went', 'match', 'almost', 'empti', 'get', 'e...",18
2654,2654,18,0.2823,"massag, recommend, highli, servic, staff, spa,...","[amaz, massag, experi, servic, outstand, nice,...",3D Museum Art in Paradise Danang,Showing results 201-210 of 211,38 contributions,Low entrance price n good visit for 2-3 hours ...,https://www.tripadvisor.co.uk/ShowUserReviews-...,Went as we had abit of time in the afternoon s...,Written 29 March 2023,Mar 2023 • Coup

In [ ]:
num_topics = 20
vectorizer = CountVectorizer()

In [ ]:
time = [1, 2, 3, 4]

# Create an empty dictionary to store the data slices dataframes
time_slices = {}

# Split the DataFrame based on the time slices
for t in time:
    time_slices[t] = new[new['timeslices'] == t]

# Print the split DataFrames
for key, value in time_slices.items():
    print(f"Time slice {key}:\n", value, "\n")

Time slice 1:
       Document_No  Dominant_Topic  Topic_Perc_Contrib  \
0               0              17              0.2497   
1               1              11              0.1965   
2               2              11              0.1926   
3               3              17              0.3700   
4               4              17              0.3318   
...           ...             ...                 ...   
2653         2653              13              0.2265   
2654         2654              13              0.2148   
2655         2655              13              0.2069   
2656         2656              11              0.1848   
2657         2657              13              0.2516   

                                               Keywords  \
0     guid, tour, inform, trip, french, super, frien...   
1     place, visit, recommend, highli, beauti, food,...   
2     place, visit, recommend, highli, beauti, food,...   
3     guid, tour, inform, trip, french, super, frien...   
4    

test

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import numpy as np
import pandas as pd

# Function to get top N words for each class
def get_top_words(matrix, feature_names, top_n=10):
    top_words = {}
    for i, row in enumerate(matrix):
        top_indices = row.argsort()[-top_n:][::-1]
        top_words[i] = [feature_names[idx] for idx in top_indices]
    return top_words

# Loop through each month in time_slice
for key, dataframe in time_slices.items():
    if dataframe.empty:
        print(f"No data found for {key}. Skipping.")
        continue

    # Sample data
    documents = dataframe['text_stm']

     # Define classes for the documents
    classes = dataframe['Dominant_Topic']

    # Calculate term frequency
    X = vectorizer.fit_transform(documents)

    # Calculate CTF
    class_counts = np.bincount(classes)
    ctf = np.zeros_like(X.toarray(), dtype=float)
    for i, class_id in enumerate(classes):
        ctf[class_id] += X.toarray()[i] / class_counts[class_id]

    # Calculate IDF
    transformer = TfidfTransformer(smooth_idf=True, use_idf=True)
    transformer.fit(X)
    idf = transformer.idf_

    # Calculate CTF-IDF
    ctf_idf = ctf * idf

        # Extract feature names (words) from the vectorizer
    feature_names = vectorizer.get_feature_names_out()

    # Get top 10 words for each class based on CTF-IDF values
    top_words_per_class = get_top_words(ctf_idf, feature_names, top_n=10)

    # Convert to a dataframe
    top_words_per_class_df = pd.DataFrame(top_words_per_class)
    # Just the first 8 columns of top_words_per_class_df
    top_words_per_class_df = top_words_per_class_df.iloc[:, :40] # <--- Change the number of columns here
    num_topics = top_words_per_class_df.shape[1]

    # Rename top_words_per_class_df columns to topic_0, topic_1, topic_2, topic_3, topic_4, topic_5, topic_6, topic_7
    top_words_per_class_df.columns = [f'topic_{i}' for i in range(num_topics)]

    topics = [f'topic_{i}' for i in range(num_topics)]  # This will generate ['topic_0', 'topic_1', ..., 'topic_7']

    # Add in 'topic_' before the string in Dominant_topic column
    dataframe['Dominant_Topic'] = 'topic_' + dataframe['Dominant_Topic'].astype(str)

    for topic in topics:
        list_values = top_words_per_class_df[topic]
        comma_separated_string = ', '.join(list_values)

        # Find rows in df where 'Dominant_topic' matches the current topic and assign the comma_separated_string to 'new_words' column
        dataframe.loc[dataframe['Dominant_Topic'] == topic, 'new_words'] = comma_separated_string

dtm_model = pd.concat(time_slices.values(), ignore_index=True)



No data found for 2. Skipping.
No data found for 3. Skipping.
No data found for 4. Skipping.


In [ ]:
viz = dtm_model.groupby(['Dominant_Topic', "Keywords", 'DATE', 'new_words']).size().reset_index(name='counts')
viz

,Dominant_Topic,Keywords,DATE,new_words,counts
0,topic_0,"walk, street, servic, room, mainli, busi, citi...",2023-02-01,"well, modern, clean, maintain, via, car, cabl,...",1
1,topic_0,"walk, street, servic, room, mainli, busi, citi...",2023-04-01,"knew, litter, magnific, touristfocus, vandal, ...",1
2,topic_1,"differ, south, district, live, qualiti, restau...",2023-01-01,"sunlight, strenuou, far, fantast, point, asian...",1
3,topic_1,"differ, south, district, live, qualiti, restau...",2023-02-01,"come, lift, special, buddhist, new, noth, foot...",1
4,topic_1,"differ, south, district, live, qualiti, restau...",2023-04-01,"boat, market, buy, sell, wholesal, transport, ...",1
5,topic_11,"place, visit, recommend, highli, beauti, food,...",2023-01-01,"danang, see, ladi, buddha, dragon, beauti, lot...",21
6,topic_11,"place, visit, recommend, highli, beauti, food,...",2023-02-01,"bridg, great, night, everyth, good, cabl, car,...",31
7,topic_11,"place, visit, recommend, highli, beauti, food,...",2023-03-01,"cabl, car, get, go, park, bridg, cave, place, ...",48
8,topic_11,"place, visit, recommend, highli, beauti, food,...",2023-04-01,"park, car, hill, bridg, cabl, ride, go, place,...",38
9,topic_11,"place, visit, recommend, highli, beauti, food,...",2023-05-01,"bridg, cabl, ride, car, get, way, place, golde...",29


In [ ]:
colors = ['rgb(49,130,189)', 'rgb(67,67,67)', 'rgb(189,189,189)', 'rgb(95,70,144)', 'rgb(115,115,115)', 'rgb(217,217,217)', 'rgb(150,150,150)', 'rgb(56,166,165)']


fig = px.area(viz, x="DATE", y="counts", color="Keywords", hover_name='new_words', line_group="Dominant_Topic", color_discrete_sequence=px.colors.qualitative.Antique, height=600)
fig.update_traces(textposition="top right", textfont_color="black")

#

fig.update_traces(marker=dict(size=6))

fig.update_traces(textposition='top center')

fig.update_layout(legend=dict(
orientation="h",
yanchor="bottom",
y=1.02,
xanchor="right",
x=1
))

"""

fig.add_annotation(
        x='2022-08-22 18:36:00.000', y=9, xref="x", yref="y", text="exhibit, suffer, audial", showarrow=True, font=dict(color="black"), align="center", ax=-10, ay=-50, bordercolor="white", borderwidth=2, borderpad=4, bgcolor="white", opacity=0.8)
"""
fig.update_layout(showlegend=True)

fig.show()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import numpy as np

# Sample data
documents = [
    "apple orange banana",
    "apple orange",
    "apple banana",
    "apple orange orange",
    "banana orange apple"
]

# Define classes for the documents
classes = [1, 1, 3, 3, 3]

# Calculate term frequency
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(documents)

# Calculate CTF
class_counts = np.bincount(classes)
ctf = np.zeros_like(X.toarray(), dtype=float)
for i, class_id in enumerate(classes):
    ctf[class_id] += X.toarray()[i] / class_counts[class_id]

# Calculate IDF
transformer = TfidfTransformer(smooth_idf=True, use_idf=True)
transformer.fit(X)
idf = transformer.idf_

# Calculate CTF-IDF
ctf_idf = ctf * idf

print("CTF-IDF:")
print(ctf_idf)

CTF-IDF:
[[0.         0.         0.        ]
 [1.         0.70273255 1.18232156]
 [0.         0.         0.        ]
 [1.         0.93697674 1.18232156]
 [0.         0.         0.        ]]


In [ ]:
import pandas as pd
import random
import string

# Function to generate a random string of a given length
def random_string(length):
    return ''.join(random.choice(string.ascii_letters + string.digits) for _ in range(length))

# Number of rows and columns
n_rows = 10
n_cols = 5

# Create the dataframe
df = pd.DataFrame({
    f'column_{i}': [random_string(10) for _ in range(n_rows)] for i in range(n_cols)
})

print(df)

     column_0    column_1    column_2    column_3    column_4
0  ocQ2JtGrXD  BNSkzdO4NT  YVRU3qC224  oP0pgAQuzu  iVDa0gWA3Z
1  zpsGX20Jlo  Dn1tl92LUp  xbvxbwG1PT  KZWk1WDCQb  qa01UNa6pD
2  cQEMtMOfWo  oVD3U9Lb3Q  8GcQcHaEBq  251WWLxcWs  hQZAPOPtgK
3  R3gFfZhREA  eUPx3kbmiX  TrwiUYfuxN  j9NuKbRQa2  ipzqOBkeUg
4  upRF2CEqe3  uqqLs9XXcu  LfpfpUzzdh  IndlvoEbWl  vEe4B8Mxvp
5  ty6w2wfdEH  HTZasE2Nrj  kDVc50A7Ou  BXJXjSZYAW  ICNiD20SUd
6  m81vsHAefR  haEtYOLxIE  LTFOWkofOh  piAKr9iAzd  mmQZHBbETC
7  DZuMXD4YcW  CGJYnHxeVT  ACQRbBlB6f  Qy8wMa8G4I  Fg6K6zTxcJ
8  XZ4vc48D8o  lLrPsGF2pE  6WcdtfgwC8  KdhhU9lsTo  d2fjOKqQQT
9  592LYSo0Hc  xIOVTRQSuU  wRDhfN816e  8Ks7oj3b8G  BqpCWN1kqD


In [ ]:
new

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text,merge_x,links_destination,links_page,links_n_contributions,links_n_review_title,...,links_n_date,city,links,DATE,text_t,text_lmt,text_stm,.len_text,timeslices,merge_y
0,0,18,0.2328,"food, kept, fire, book, district, water, perfe...","[famili, book, trip, market, via, organis, fan...",0,Cai Rang Floating Market,"Showing results 201-210 of 1,186",5 contributions,Best trip ever,...,Jun 2023 • Family,Can Tho,Family,2023-06-01,"[family, booked, trip, market, via, organised,...","[family, booked, trip, market, via, organised,...","['famili', 'book', 'trip', 'market', 'via', 'o...",97,1,0
1,1,18,0.1825,"food, kept, fire, book, district, water, perfe...","[visit, cai, rang, float, market, join, cruis,...",1,Cai Rang Floating Market,"Showing results 201-210 of 1,186",9 contributions,Cai Rang floating market,...,May 2023 • Couples,Can Tho,Couples,2023-05-01,"[visited, cai, rang, floating, market, join, c...","[visited, cai, rang, floating, market, join, c...","['visit', 'cai', 'rang', 'float', 'market', 'j...",17,1,1
2,2,3,0.2828,"place, visit, recommend, highli, understand, m...","[market, interest, mani, fruit, boat, float, c...",2,Cai Rang Floating Market,"Showing results 201-210 of 1,186",1 contribution,Wonderful Experience,...,Apr 2023 • Couples,Can Tho,Couples,2023-04-01,"[market, interesting, many, fruit, boats, floa...","[market, interesting, many, fruit, boat, float...","['market', 'interest', 'mani', 'fruit', 'boat'...",31,1,2
3,3,8,0.2511,"market, differ, local, stall, south, night, se...","[guid, pick, hotel, got, float, market, around...",3,Cai Rang Floating Market,"Showing results 201-210 of 1,186",1 contribution,Just amazing!!!,...,May 2023 • Friends,Can Tho,Friends,2023-05-01,"[guide, picked, hotel, got, floating, market, ...","[guide, picked, hotel, got, floating, market, ...","['guid', 'pick', 'hotel', 'got', 'float', 'mar...",77,1,3
4,4,19,0.3407,"year, clean, buy, ago, souvenir, old, famili, ...","[mr, toan, boutiqu, homestay, took, us, boat, ...",4,Cai Rang Floating Market,"Showing results 201-210 of 1,186",135 contributions,Good market experience,...,Mar 2023 • Couples,Can Tho,Couples,2023-03-01,"[mr, toan, boutique, homestay, took, us, boat,...","[mr, toan, boutique, homestay, took, u, boat, ...","['mr', 'toan', 'boutiqu', 'homestay', 'took', ...",66,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2653,2653,3,0.1463,"place, visit, recommend, highli, understand, m...","[complex, build, central, pagoda, make, nice, ...",2653,3D Museum Art in Paradise Danang,Showing results 201-210 of 211,104 contributions,Good place for instagrammers and small kids,...,Mar 2023 • Family,Da Nang,Family,2023-03-01,"[went, match, almost, empty, get, enough, spac...","[went, match, almost, empty, get, enough, spac...","['went', 'match', 'almost', 'empti', 'get', 'e...",18,1,2653
2654,2654,3,0.1502,"place, visit, recommend, highli, understand, m...","[interest, templ, hcmc, introduc, hostel, rece...",2654,3D Museum Art in Paradise Danang,Showing results 201-210 of 211,38 contributions,Low entrance price n good visit for 2-3 hours ...,...,Mar 2023 • Couples,Da Nang,Couples,2023-03-01,"[went, abit, time, afternoon, since, location,...","[went, abit, time, afternoon, since, location,...","['went', 'abit', 'time', 'afternoon', 'sinc', ...",75,1,2654
2655,2655,3,0.1296,"place, visit, recommend, highli, understand, m...","[thought, buddhist, shrine, appar, mixtur, chi...",2655,3D Museum Art in Paradise Danang,Showing results 201-210 of 211,1 contribution,Place Yourself Into The Action,...,Mar-23,Da Nang,None,2023-03-01,"[art, museum, open, entry, us, exhibit, spans,...","[art, museum, open, entry, u, exhibit, span, m...","['art', 'museum', 'open', 'entri', 'us', 'exhi...",48,1,2655
2656,2656,4,0.1463,"beauti, room, architectur, seen, mainli, view,...","[best, mayb, pagoda, realli, worth, saigon, sa...",2656,3D Museum Art in Paradise Danang,

In [ ]:
column_0_list = new['text_stm'].tolist()
print(column_0_list)

["['famili', 'book', 'trip', 'market', 'via', 'organis', 'fantast', 'trip', 'us', 'love', 'knowledg', 'tourguid', 'firstli', 'clare', 'came', 'hotel', 'pick', 'bring', 'us', 'sampan', 'boat', 'spent', 'minut', 'get', 'market', 'mani', 'fruit', 'boat', 'guid', 'bought', 'us', 'two', 'pineappl', 'delici', 'like', 'someon', 'put', 'sugar', 'insid', 'also', 'got', 'coffe', 'noodl', 'soupl', 'breakfast', 'best', 'breakfast', 'never', 'afterthat', 'cruis', 'small', 'wild', 'canal', 'villag', 'realli', 'beauti', 'mani', 'water', 'hyacinth', 'wate', 'coconut', 'surround', 'best', 'part', 'trip', 'met', 'local', 'peopl', 'pick', 'delic', 'fruit', 'garden', 'us', 'highli', 'recommend', 'mangoateen', 'star', 'appl', 'tri', 'final', 'stop', 'cocoa', 'factori', 'noodl', 'factori', 'awesom', 'highli', 'recommen', 'take', 'trip', 'whatsapp', 'realli', 'love', 'knowledg', 'trip', 'comeplet', 'authent']", "['visit', 'cai', 'rang', 'float', 'market', 'join', 'cruis', 'caibecantho', 'mekong', 'eye', 'cru

# testing

test

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import numpy as np

# Sample data
documents = time_slice['june']['text_stm']

# Define classes for the documents
classes = time_slice['june']['Dominant_Topic']

# Calculate term frequency
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(documents)

# Calculate CTF
class_counts = np.bincount(classes)
ctf = np.zeros_like(X.toarray(), dtype=float)
for i, class_id in enumerate(classes):
    ctf[class_id] += X.toarray()[i] / class_counts[class_id]

# Calculate IDF
transformer = TfidfTransformer(smooth_idf=True, use_idf=True)
transformer.fit(X)
idf = transformer.idf_

# Calculate CTF-IDF
ctf_idf = ctf * idf

In [ ]:
# Extract feature names (words) from the vectorizer
feature_names = vectorizer.get_feature_names_out()

# Function to get top N words for each class
def get_top_words(matrix, feature_names, top_n=10):
    top_words = {}
    for i, row in enumerate(matrix):
        top_indices = row.argsort()[-top_n:][::-1]
        top_words[i] = [feature_names[idx] for idx in top_indices]
    return top_words

# Get top 10 words for each class based on CTF-IDF values
top_words_per_class = get_top_words(ctf_idf, feature_names, top_n=10)

In [ ]:
top_words_per_class_df = pd.DataFrame(top_words_per_class)
#to a dataframe
top_words_per_class_df = pd.DataFrame(top_words_per_class)
#just the first 8 columns of top_words_per_class_df
top_words_per_class_df = top_words_per_class_df.iloc[:, :8]
top_words_per_class_df

#rename top_words_per_class_df columns to topic_0, topic_1, topic_2, topic_3, topic_4, topic_5, topic_6, topic_7
top_words_per_class_df.columns = [f'topic_{i}' for i in range(num_topics)]

In [ ]:
topics = [f'topic_{i}' for i in range(num_topics)]  # This will generate ['topic_0', 'topic_1', ..., 'topic_7']

#add in 'topic_' before the string in Dominant_topic column
time_slice['june']['Dominant_Topic'] = 'topic_' + time_slice['june']['Dominant_Topic'].astype(str)

for topic in topics:
    list_values = top_words_per_class_df[topic]
    comma_separated_string = ', '.join(list_values)

    # Find rows in df where 'Dominant_topic' matches the current topic and assign the comma_separated_string to 'new_words' column
    time_slice['june'].loc[time_slice['june']['Dominant_Topic'] == topic, 'new_words'] = comma_separated_string

/var/folders/rh/dpkch0xj1dx4bbcqs_mnb3qr0000gn/T/ipykernel_20731/3518073697.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/rh/dpkch0xj1dx4bbcqs_mnb3qr0000gn/T/ipykernel_20731/3518073697.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
time_slice['june']

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text,links_destination,links_page,links_n_contributions,links_n_review_title,links_n_review_title_url,...,links_n_date_written,links_n_date,city,links,DATE,text_t,text_lmt,text_stm,.len_text,new_words
0,0,topic_5,0.4860,"war, tunnel, histori, museum, visit, templ, pe...","['famili', 'book', 'trip', 'market', 'via', 'o...",Cai Rang Floating Market,"Showing results 201-210 of 1,186",5 contributions,Best trip ever,https://www.tripadvisor.co.uk/ShowUserReviews-...,...,Written 18 July 2023,Jun 2023 • Family,Can Tho,Family,2023-06-01,"[family, booked, trip, market, via, organised,...","[family, booked, trip, market, via, organised,...","['famili', 'book', 'trip', 'market', 'via', 'o...",97,"war, tunnel, museum, histori, vietnam, interes..."
1,1,topic_6,0.5164,"massag, staff, recommend, servic, relax, spa, ...","['spa', 'experi', 'amaz', 'like', 'coupl', 'ro...",L'essence De La Vie Spa,Showing results 201-210 of 603,1 contribution,Amazing service!,https://www.tripadvisor.co.uk/ShowUserReviews-...,...,Written 30 June 2023,Jun 2023 • Couples,Hanoi,Couples,2023-06-01,"[spa, experience, amazing, like, couple, room,...","[spa, experience, amazing, like, couple, room,...","['spa', 'experi', 'amaz', 'like', 'coupl', 'ro...",15,"massag, servic, staff, spa, recommend, experi,..."
2,2,topic_6,0.6040,"massag, staff, recommend, servic, relax, spa, ...","['excel', 'custom', 'servic', 'begin', 'end', ...",La Belle Vie Spa,Showing results 201-210 of 799,1 contribution,Great massage,https://www.tripadvisor.co.uk/ShowUserReviews-...,...,Written 15 June 2023,Jun 2023 • Family,Hanoi,Family,2023-06-01,"[excellent, customer, service, beginning, end,...","[excellent, customer, service, beginning, end,...","['excel', 'custom', 'servic', 'begin', 'end', ...",10,"massag, servic, staff, spa, recommend, experi,..."
3,3,topic_2,0.8742,"place, visit, price, walk, manag, book, old, q...","['market', 'great', 'hype', 'full', 'counterfe...",Ben Thanh Market,"Showing results 201-210 of 9,430",194 contributions,A much hyped but yet to see,https://www.tripadvisor.co.uk/ShowUserReviews-...,...,Written 28 June 2023,Jun 2023 • Family,Ho Chi Minh City,Family,2023-06-01,"[market, great, hype, full, counterfeit, produ...","[market, great, hype, full, counterfeit, produ...","['market', 'great', 'hype', 'full', 'counterfe...",41,"place, cave, visit, old, ride, time, walk, str..."
4,4,topic_6,0.7199,"massag, staff, recommend, servic, relax, spa, ...","['excel', 'servic', 'perfect', 'messag', 'real...",La Belle Vie Spa,Showing results 201-210 of 799,4 contributions,Amazing!,https://www.tripadvisor.co.uk/ShowUserReviews-...,...,Written 15 June 2023,Jun 2023 • Family,Hanoi,Family,2023-06-01,"[excellent, service, perfect, message, really,...","[excellent, service, perfect, message, really,...","['excel', 'servic', 'perfect', 'messag', 'real...",13,"massag, servic, staff, spa, recommend, experi,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349,349,topic_2,0.6033,"place, visit, price, walk, manag, book, old, q...","['hectic', 'parti', 'citi', 'head', 'must', 's...",Old Quarter,"Showing results 201-210 of 21,055",88 contributions,Must visit,https://www.tripadvisor.co.uk/ShowUserReviews-...,...,Written 11 June 2023,Jun 2023 • Couples,Hanoi,Couples,2023-06-01,"[hectic, party, city, head, must, swivel, time...","[hectic, party, city, head, must, swivel, time...","['hectic', 'parti', 'citi', 'head', 'must', 's...",23,"place, cave, visit, old, ride, time, walk, str..."
350,350,topic_7,0.4480,"show, view, variou, dri, citi, life, bridg, wa...","['awesom', 'place', 'spend', 'half', 'day', 's...",The Marble Mountains,"Showing results 201-210 of 5,308","1,355 contributions",great workout,https://www.tripadvisor.co.uk/ShowUserReviews-...,...,Written 8 July 2023,Jun-23,Da Nang,None,2023-06-01,"[awesome, place, spend, half, day, sure, authe...","[awesome, place, spend, half, day, sure, authe...